This week we're going to look at implementations of core data structures in Python.

## Linked Lists vs. Arrays

Arrays are implemented as blocks of contiguous memory.  This means looking up the 0th element takes the same amount of time as the 1,000,00th.  This is an `O(1)` operation, since the time it takes to look up an element is constant.

Python's `list` type is internally implemented as an array.

How long does it take to add an element to the beginning of a list? The end?

An alternative way to store sequential items is by using a linked list.

Linked lists store individual elements and a pointer to the next element.  This means that looking up the Nth element requires traversing the entire list.  (We can also doubly-link our lists, allowing us to traverse in either direction, gaining some speed at the cost of a bit more storage.)

![](https://upload.wikimedia.org/wikipedia/commons/thumb/6/6d/Singly-linked-list.svg/408px-Singly-linked-list.svg.png)

Linked lists can grow without bound, each new node can be allocated on the fly.

### Questions

In [ ]:
class Node:
    def __init__(self, value, _next=None):
        self.value = value
        self.next = _next


class LinkedList:
    def __init__(self):
        self.root = None

    def add(self, value):
        if self.root is None:
            # first value: special case
            self.root = Node(value)
        else:
            cur = self.root
            # traverse to end of list
            while cur.next:
                cur = cur.next
            # drop a new node at the end of list
            cur.next = Node(value)

    def __str__(self):
        s = ""
        cur = self.root
        while cur:
            s += f"[{cur.value}] -> "
            cur = cur.next
        s += "END"
        return s


In [ ]:
ll = LinkedList()
ll.add(1)
ll.add(3)
ll.add(5)
ll.add(7)
print(ll)


### Optimizations

Doubly linked lists, and more complicated internal pointer structures can lead to increased performance at cost of more memory/complexity.

`collections.deque` is a doubly linked list implementation in Python.

## Stack

A stack is a last-in-first-out (LIFO) data structure that needs to primarily serve two operations: push, and pop.



In [ ]:
class Stack:
    def __init__(self):
        self._data = []

    def push(self, item):
        # remember: adding/removing at the end of the list is faster than the front
        self._data.append(item)

    def pop(self):
        return self._data.pop()

    def __len__(self):
        return len(self._data)

    def __str__(self):
        return " TOP -> " + "\n        ".join(
            f"[ {item} ]" for item in reversed(self._data)
        )


In [ ]:
s = Stack()
s.push(1)
s.push(2)
s.push(3)
print(s)
print("removed item", s.pop(), "stack is now", len(s))
print(s)
print("removed item", s.pop(), "stack is now", len(s))
print(s)


### Performance

Insertion: `O(1)`
Removal: `O(1)`
Lookup: `O(n)` (not a typical operation on a stack)

### Usage

"The Stack"

Sometimes when we're writing code we talk about "the stack", which is the stack of functions we're in & their scopes.

```python

def f():
    ...
    
    
def g():
    if ...:
        f()
    else:
        ...

def h():
    y = g()
    ...
```

When we call h(), it is added to the call stack, then g is added, and f is added on top.  We return from these functions in LIFO order, f() exits, then g(), then h().

## Queue

A queue is a first-in-first-out (FIFO) data structure that adds items to one end, and removes them from the other.



In [ ]:
class ArrayQueue:
    def __init__(self, _iterable=None):
        if _iterable:
            self._data = list(_iterable)
        else:
            self._data = []

    def push(self, item):
        # adding to the end of the list is faster than the front
        self._data.append(item)

    def pop(self):
        # only change from `Stack` is we remove from the other end
        # this can be slower, why?
        return self._data.pop(0)

    def __len__(self):
        return len(self._data)

    def __repr__(self):
        return " TOP -> " + "\n        ".join(
            f"[ {item} ]" for item in reversed(self._data)
        )


In [ ]:
from collections import deque


class DequeQueue:
    def __init__(self, _iterable=None):
        if _iterable:
            self._data = deque(_iterable)
        else:
            self._data = deque()

    def push(self, item):
        self._data.append(item)

    def pop(self):
        return self._data.popleft()

    def __len__(self):
        return len(self._data)

    def __repr__(self):
        return " TOP -> " + "\n        ".join(
            f"[ {item} ]" for item in reversed(self._data)
        )


### Usage

Where do we see queues in the real world? Computing?

## Performance Testing

We can try to measure performance something takes by measuring how much time has passed.

```python
import time

before = time.time()
# do something
after = time.time()
elapsed = before - after
```

Issue is that in practice, times involved are miniscule, and other events on the system will overshadow differences.

In [ ]:
import time


def print_elapsed(func):
    def newfunc(*args, **kwargs):
        before = time.time()
        retval = func(*args, **kwargs)
        elapsed = time.time() - before
        print(f"Took {elapsed} sec to run {func.__name__}")

    return newfunc


In [ ]:
@print_elapsed
def testfunc(QueueCls):
    queue = QueueCls()
    for item in range(100):
        queue.push(item)
    while queue:
        queue.pop()


In [ ]:
testfunc(ArrayQueue)

In [ ]:
testfunc(DequeQueue)

In [ ]:
# run again, see how they differ
testfunc(ArrayQueue)
testfunc(DequeQueue)

The differences are just too small to be sure.  We need to run our code many more times.

Python has a built in module for this, `timeit`.

```python
import timeit

timeit.timeit(stmt='pass', setup='pass', timer=<default timer>, number=1000000, globals=None)

# for more: https://docs.python.org/3/library/timeit.html
```

In [ ]:
import timeit

number = 1_000_000

elapsed = timeit.timeit(
    "queue.push(1)",
    setup="queue = QueueCls()",
    globals={"QueueCls": ArrayQueue},
    number=number,
)
elapsed2 = timeit.timeit(
    "queue.push(1)",
    setup="queue = QueueCls()",
    globals={"QueueCls": DequeQueue},
    number=number,
)
print(f"{number}x ArrayQueue.push, took", elapsed)
print(f"{number}x DequeQueue.push, took", elapsed2)
print(f"DequeQueue is {(elapsed-elapsed2) / elapsed * 100:.3f}% faster")

In [ ]:
number = 10_000

elapsed = timeit.timeit(
    "queue.pop()",
    setup="queue = QueueCls([0] * 1000000)",
    globals={"QueueCls": ArrayQueue},
    number=number,
)
elapsed2 = timeit.timeit(
    "queue.pop()",
    setup="queue = QueueCls([0] * 1000000)",
    globals={"QueueCls": DequeQueue},
    number=number,
)
print(f"{number}x ArrayQueue.pop, took", elapsed)
print(f"{number}x DequeQueue.pop, took", elapsed2)
print(f"DequeQueue is {(elapsed-elapsed2) / elapsed * 100:.3f}% faster")


### Queue Performance

| Operation | ArrayQueue | DequeQueue |
| --------- | ---------- | ---------- |
| push      | O(1)       | O(1)       |
| pop       | O(n)       | O(1)       |


## Hash Tables

A hash table is a collection that maps keys to values.  Python's `dict` is an implementation of a hash table.

For the course project, you will be implementing a hash table without using `dict`.

### Hash Table Performance

| Operation | Average | Worst Case | 
| --------- | ---- | ---------- |
| lookup    | O(1) | O(n) |
| insert    | O(1) | O(n) |
| delete    | O(1) | O(n) |

Note: These are average case, as we'll see, depending on implementation, worst case can be much worse.

A key property for hash tables is that we **do not need to linearly search through them for our data**.

If you find yourself scanning every element in a hash table, you're doing something wrong.

### Example 1

Let's first model a simple hashtable with fixed capacity of 10.  For simplicity we'll stick to string keys.

```
[    ][    ][    ][    ][    ][    ][    ][    ][    ][    ]
```

When we get a key-value pair, we need to assign it a bucket.

How can we write a function that takes a string and assigns it to a bucket?

1. Turn string into a number. **Hash Function**
2. Take (number % capacity)

In [ ]:
def strhash(key):
    # ord converts a character to it's numeric representation
    #   ord("A") == 65
    #   ord("z") == 122
    # etc.
    return sum(ord(letter) for letter in key)


In [ ]:
strhash("bear")

In [ ]:
strhash("fox")


In [ ]:
class Hashtable:
    def __init__(self, capacity=10):
        self._table = [None] * capacity
        self.capacity = capacity

    def __setitem__(self, key, value):
        index = strhash(key) % self.capacity
        self._table[index] = value

    def __getitem__(self, key):
        index = strhash(key) % self.capacity
        value = self._table[index]
        if value:
            return value
        else:
            raise KeyError(key)

    def display(self):
        print(f"Capacity = {self.capacity}")
        for idx, elem in enumerate(self._table):
            if elem:
                print(f"{idx}: {elem}")


In [ ]:
h = Hashtable()
h["bear"] = 3
h["fox"] = 12
h.display()

In [ ]:
print(h["bear"])

In [ ]:
strhash("bear") == strhash("been")  # different word, same hash!

In [ ]:
h = Hashtable()
h["bear"] = 33
h["been"] = 12
h.display()


### Linear Probing

One solution is to just walk forward in the storage list, until we find an empty space.

Either way, we'll need to start storing the key as well.   Let's revise our class:

In [ ]:
class Hashtable:
    def __init__(self, capacity=100):
        self._table = [None] * 100
        self.capacity = capacity

    def __setitem__(self, key, value):
        index = strhash(key) % self.capacity
        while self._table[index] is not None:
            index += 1
            # Handling wrap-around omitted for brevity

        # we now store the key and value
        self._table[index] = (key, value)

    def __getitem__(self, key):
        index = strhash(key) % self.capacity

        # walk forward until we either reach the item or an empty space
        while self._table[index] is not None:
            if self._table[index][0] == key:
                return self._table[index][1]
            index += 1
            # Handling wrap-around omitted for brevity

        # if code got here, the item wasn't in the list
        raise KeyError(key)

    def display(self):
        print(f"Capacity = {self.capacity}")
        for idx, elem in enumerate(self._table):
            if elem:
                print(f"{idx}: {elem}")


In [ ]:
h = Hashtable()
h["bear"] = 33
h["been"] = 12
h.display()

In [ ]:
print(h["bear"])
print(h["been"])


### Separate Chaining

Another solution is to use a linked list to store multiple items in the same bucket.

![](images/hashtable-chaining.png)

This is the approach we're asking you to use for the course project.

**What happens if we have a lot of items in the same bucket?**


### Better Hash Function

Ideally a hash function will evenly distribute values across the collection.

A common pattern is to use a polynomial hash function.

$$h(x_0, ..., x_n) = (\sum_{i=0}^{k-1}{c_ip^i})\mod{m}$$

Where:
- $x_0...x_i $ is the sequence
- $k = len(x)$
- $c_i$ is the numeric value of the character $x_i$ ($ord(x$ in Python)
- $p$ is an arbitrary constant.
- and $m$ is the size of the collection.

```python
    def _hash(self, key):
        """
        This method takes in a string and returns 
        an integer value between 0 and self.capacity.

        This particular hash function uses 
        Horner's rule to compute a large polynomial.
        """
        val = 0
        for letter in key:
            val = self.P_CONSTANT * val + ord(letter)
        return val % self.capacity
```

### Questions

* Linear probing vs. separate chaining? Other approaches?
* What if our hash function wasn't reliable?

### Rehashing

As we add more items to our hash table, we'll eventually run out of space.  We can increase the capacity of our table, but we'll need to rehash all of our existing items.

Because which bucket we choose depends on `hash(item) % capacity` items would end up in different buckets if we change capacity.

This means when we resize, we need to rehash all of our items.

A common pattern is to double capacity when the table is ~50% full.

## Trees

Somewhat similar to lists but instead of each node having a "next", nodes can have multiple children.


### Binary Search Tree

In [ ]:
class Node:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def __str__(self):
        return f"({self.value}, {self.left}, {self.right})"


class BST:
    def __init__(self, iterable=None):
        self.root = None
        if iterable:
            for item in iterable:
                self.add_item(item)

    def add_item(self, newval):
        # special case: first item
        if self.root is None:
            self.root = Node(newval)
        else:
            parent = self.root
            # traverse until we find room in the tree
            while True:
                if newval < parent.value:
                    if parent.left:
                        parent = parent.left
                    else:
                        parent.left = Node(newval)
                        break
                else:
                    if parent.right:
                        parent = parent.right
                    else:
                        parent.right = Node(newval)
                        break


def print_infix(node):
    """prints items in sorted order"""
    if node.left:
        print_infix(node.left)
    print(node.value)
    if node.right:
        print_infix(node.right)


Tree traversal is inherently recursive, so we'll use a recursive function to print the tree in sorted order.

Most tree algorithms will operate on the left & right subtrees the same way, so we can write a recursive function that takes a node and calls itself on the left & right subtrees.

In [ ]:
tree = BST()
tree.add_item("Fox")
tree.add_item("Wolf")
tree.add_item("Bear")
tree.add_item("Raccoon")
tree.add_item("Rabbit")

In [ ]:
print_infix(tree.root)

#### Aside: defaultdict

```python
# common pattern:
if key not in dct:
    dct[key] = []
dct[key].append(element)
```

We can instead use `collections.defaultdict`:

In [ ]:
from collections import defaultdict

# give defaultdict a function that it will use to generate missing keys
dd = defaultdict(set)

print(dd["newkey"])
print(dd)

dd["newset"].add(1)  # can add to set without ensuring it exists

## Graphs

![](https://www.simplilearn.com/ice9/free_resources_article_thumb/Graph%20Data%20Structure%20-%20Soni/what-is-graphs-in-data-structure.png)

In [ ]:
class Graph:
    def __init__(self):
        # create a dictionary where every string maps to a set of strings
        self.edges = defaultdict(set)

    def add_edge(self, node1, node2):
        # add in both directions, could alter for directed graph
        self.edges[node1].add(node2)
        self.edges[node2].add(node1)

    def find_path(self, from_node, to_node, seen=None):
        if not seen:
            seen = set()

        if to_node in self.edges[from_node]:
            return (from_node, to_node)
        else:
            for sibling in self.edges[from_node] - seen:
                return (from_node,) + self.find_path(
                    sibling, to_node, seen | set(sibling)
                )
            # return self.find_path(


In [ ]:
g = Graph()
g.add_edge("A", "D")
g.add_edge("B", "D")

In [ ]:
g.find_path("A", "B")

In [ ]:
g = Graph()
g.add_edge("A", "B")
g.add_edge("B", "C")
g.add_edge("C", "D")
g.add_edge("D", "E")
g.find_path("A", "E")

### Discussion

* Graphs & Trees in the real world?
* Alternate implementations?